## Competition Objective is to detect fraud in transactions;

### Data
In this competition, you are predicting the probability that an online transaction is fraudulent, as denoted by the binary target 'isFraud'.

The data is broken into two files 'identity' and 'transaction', which are joined by 'TransactionID'.

> Note : Not all transactions have corresponding identity information.

##### Categorical Features-Transaction
* ProductCD
* emaildomain
* card1-card6
* addr1,addr2
* P_emaildomain
* R_emaildomain
* M1-M9

##### Categorical Features-Identity
* DeviceType
* DeviceInfo
* id_12-id_38

### Questions
1. What type of data we have on our data?
2. How many cols, rows, missing values we have?
3. What's the target distribution?
4. What's the transactions values distribution of fraud and no fraud transactions?
5. We have predominant fraudulent products?
6. What features or target shows some interesting patterns?
7. A lot of more questions that will raise throughout the exploration.

### Importing necessary libraries

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing
import scipy as sp
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

# Standard plotly imports
import plotly.graph_objs as go
import plotly.tools as tls
from plotly.offline import iplot, init_notebook_mode
# import cufflinks
# import cufflinks as cf
import plotly.figure_factory as ff

# Using plotly + cufflinks in offline mode
init_notebook_mode(connected=True)
# cufflinks.go_offline(connected=True)

# Preprocessing, modelling and evaluating
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_val_score,KFold
from xgboost import XGBClassifier
import xgboost as xgb

# Hyperopt modules
from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK, STATUS_RUNNING
from functools import partial

import os
import gc
print(os.listdir("../input"))

['ieee-fraud-detection']


### Importing train datasets

In [2]:
df_id=pd.read_csv("../input/ieee-fraud-detection/train_identity.csv")

In [3]:
df_trans=pd.read_csv("../input/ieee-fraud-detection/train_transaction.csv")

In [4]:
def resumetable(df):
    print(f"Dataset Shape: {df.shape}")
    summary=pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary=summary.reset_index()
    summary['Name']=summary['index']
    summary=summary[['Name','dtypes']]
    summary['Missing']=df.isnull().sum().values
    summary['Uniques']=df.nunique().values
    summary['First Value']=df.loc[0].values
    summary['Second Value']=df.loc[1].values
    summary['Third Value']=df.loc[2].values
    
    for name in summary['Name'].value_counts().index:
        summary.loc[summary['Name']==name,'Entropy']=round(stats.entropy(df[name].value_counts(normalize=True),base=2),2)
        
    return summary

In [5]:
## Function to reduce DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [6]:
def CalcOutliers(df_num):
    
    # calculating mean and std of the array
    data_mean,data_std=np.mean(df_num),np.std(df_num)
    
    # setting the cut line to both higher and lower values
    cut=data_std*3
    
    # Caculating the higher and lower cut values
    lower,upper=data_mean-cut,data_mean+cut
    
    # creating an array of lower, higher and total outlier values
    outliers_lower=[x for x in df_num if x<lower]
    outliers_higher=[x for x in df_num if x>upper]
    outliers_total=[x for x in df_num if x<lower or x>upper]
    
    # array without outlier values
    outliers_removed=[x for x in df_num if x>lower and x<upper]
    
    # printing total number of values in lower cut of outliers
    print('Identified lowest outliers: %d' % len(outliers_lower))
    # printing total number of values in higher cut of outliers
    print('Identified upper outliers: %d' % len(outliers_higher))
    # printing total number of values outliers of both sides
    print('Total outliers observations: %d' % len(outliers_total))
    # printing total number of non-outlier values
    print('Non-outlier observations: %d' % len(outliers_removed))
    # percentual of outliers in points
    print('Total precentual of Outliers: ', round((len(outliers_total)/len(outliers_removed))*100,4))
    
    return

In [7]:
# Reducing memory
df_trans=reduce_mem_usage(df_trans)
df_id=reduce_mem_usage(df_id)

Mem. usage decreased to 542.35 Mb (69.4% reduction)
Mem. usage decreased to 25.86 Mb (42.7% reduction)


### Knowing the data

In [ ]:
resumetable(df_trans)[:25]

### Target Distribution

In [ ]:
df_trans['TransactionAmt']=df_trans['TransactionAmt'].astype(float)
total=len(df_trans)
total_amt=df_trans.groupby(['isFraud'])['TransactionAmt'].sum().sum()
plt.figure(figsize=(16,6))

plt.subplot(121)
g=sns.countplot(x='isFraud',data=df_trans,)
g.set_title("Fraud Transactions Distribution \n# 0: No Fraud | 1: Fraud #",fontsize=22)
g.set_xlabel("Is Fraud?",fontsize=18)
g.set_ylabel("Count",fontsize=18)

for p in g.patches:
    height=p.get_height()
    g.text(p.get_x()+p.get_width()/2.,height+3,"{:1.2f}%".format(height/total*100),ha="center",fontsize=15)
    
    perc_amt=(df_trans.groupby(['isFraud'])['TransactionAmt'].sum())
    perc_amt=perc_amt.reset_index()
    plt.subplot(122)
    g1=sns.barplot(x='isFraud',y='TransactionAmt',dodge=True,data=perc_amt)
    g1.set_title("% Total Amount in Transaction Amt \n# 0: No Fraud | 1: Fraud #",fontsize=22)
    g1.set_xlabel("Is fraud?",fontsize=18)
    g1.set_ylabel('Total Transaction Amount Scalar',fontsize=18)
    for p in g1.patches:
        height=p.get_height()
        g1.text(p.get_x()+p.get_width()/2.,height+3,'{:1.2f}%'.format(height/total_amt*100),ha="center",fontsize=15)
plt.show()

We have 3.5% of Fraud transaction in out datasets.
I think that it would be interesting to see if the amount of percentuals is higher or lower than 3.5% of total.

### Transaction Amount Quantiles

Before ploting the Transaction Amount, let's see the quantiles of Transaction Amount.

In [ ]:
df_trans['TransactionAmt']=df_trans['TransactionAmt'].astype(float)
print("Transaction Amounts Quatiles:")
print(df_trans['TransactionAmt'].quantile([.01,.025,.1,.25,.5,.75,.9,.975,.99]))

### Ploting Transaction Amount Values Distribution

In [ ]:
plt.figure(figsize=(16,12))
plt.suptitle('Transaction Values Distribution', fontsize=22)
plt.subplot(221)
g=sns.distplot(df_trans[df_trans['TransactionAmt']<=1000]['TransactionAmt'])
g.set_title("Transaction Amount Distribution<=1000",fontsize=18)
g.set_xlabel("")
g.set_ylabel("Probability",fontsize=15)

plt.subplot(222)
g1=sns.distplot(np.log(df_trans["TransactionAmt"]))
g1.set_title("Transaction Amount (Log) Distribution",fontsize=18)
g1.set_xlabel("")
g1.set_ylabel("Probability",fontsize=15)

plt.figure(figsize=(16,12))

In [ ]:
plt.subplot(212)
g4=plt.scatter(range(df_trans[df_trans['isFraud']==0].shape[0]),
               np.sort(df_trans[df_trans['isFraud']==0]['TransactionAmt'].values),
               label="Fraud",alpha=.2)
g4=plt.scatter(range(df_trans[df_trans['isFraud']==1].shape[0]),
              np.sort(df_trans[df_trans['isFraud']==1]['TransactionAmt'].values),
              label='Fraud',alpha=.2)
g4=plt.title("ECDF \nFRAUD and NO FRAUD Transaction Amount Distribution",fontsize=15)
g4=plt.xlabel("Index")
g4=plt.ylabel("Amount Distribution",fontsize=10)
g4=plt.legend()

In [ ]:
plt.figure(figsize=(16,12))

plt.subplot(322)
g1=plt.scatter(range(df_trans[df_trans['isFraud']==0].shape[0]),
              np.sort(df_trans[df_trans['isFraud']==0]['TransactionAmt'].values),
              label='NoFraud',alpha=.2)
g1=plt.title("NO FRAUD - Transaction Amount ECDF",fontsize=18)
g1=plt.xlabel("Index")
g1=plt.ylabel("Amount Distribution",fontsize=15)

plt.suptitle('Individual ECDF Distribution',fontsize=22)

plt.show()

### Seeing the Quantiles of Fraud and No Fraud Transactions

In [ ]:
print(pd.concat([df_trans[df_trans['isFraud']==1]['TransactionAmt'].quantile([.01,.1,.25,.5,.75,.9,.99]).reset_index(),
     df_trans[df_trans['isFraud']==0]['TransactionAmt'].quantile([.01,.1,.25,.5,.75,.9,.99]).reset_index()],
     axis=1,keys=['Fraud','No Fraud']))

## Transaction Amount Outliers

In [ ]:
CalcOutliers(df_trans['TransactionAmt'])

If wer consider only values between 0 to 800, we will avoid the outliers and has more confidence in our distribution. We have 10k rows with outliers that represents 1.74% of total rows.

## Now, let's know the Product Feature

* Distribution Products
* Distribution of Frauds by Product
* Has Difference between Transaction Amounts in Products?

In [ ]:
tmp=pd.crosstab(df_trans['ProductCD'],df_trans['isFraud'],normalize='index')*100
tmp=tmp.reset_index()
tmp.rename(columns={0:'NoFraud',1:'Fraud'},inplace=True)

plt.figure(figsize=(14,10))
plt.suptitle('ProductCD Distribution',fontsize=22)

plt.subplot(221)
g=sns.countplot(x='ProductCD',data=df_trans)
# plt.legend(title='Fraud',loc='upper center',labels=['No','Yes'])

g.set_title("ProductCD Distribution",fontsize=19)
g.set_xlabel("ProductCD Name",fontsize=17)
g.set_ylabel("Count",fontsize=17)
g.set_ylim(0,500000)
for p in g.patches:
    height=p.get_height()
    g.text(p.get_x()+p.get_width()/2,height+3,'{:1.2f}%'.format(height/total*100),ha='center',fontsize=14)
    
plt.subplot(222)
g1=sns.countplot(x='ProductCD',hue='isFraud',data=df_trans)
plt.legend(title='Fraud',loc="best",labels=["No","Yes"])
gt=g1.twinx()
gt=sns.pointplot(x='ProductCD',y='Fraud',data=tmp,color='black',order=['W','H','C','S','R'],lengend=False)
gt.set_ylabel("% of Fraud Transactions",fontsize=16)

g1.set_title("Product CD by Target(isFraud)",fontsize=19)
g1.set_xlabel("ProductCD Name",fontsize=17)
g1.set_ylabel("Count",fontsize=17)

In [ ]:
plt.subplot(212)
g3=sns.boxenplot(x='ProductCD',y='TransactionAmt',hue='isFraud',data=df_trans[df_trans['TransactionAmt']<=2000])
g3.set_xlabel('ProductCD Name',fontsize=17)
g3.set_ylabel("Transaction Values",fontsize=17)

plt.subplots_adjust(hspace=0.6,top=0.85)

plt.show()

W,C, and R are the most frequent values.
We can note that in W,H and R the distribution of Fraud values are slightly higher than the Non-Fraud Transactions.

## Card Features
* Card features are categoricals.
* Let's understand the distribution of values.
* What's the difference in transactions and % of Fraud for each values in these features.
* Card features has 6 columns, and 4 of them seems to be numericals, so let's see the quantiles and distributions.

In [ ]:
## Knowing the Card Features
resumetable(df_trans[['card1','card2','card3','card4','card5','card6']])

Card2-Card6 has some missing values. We will need to due with it later.

## Numerical Feature Card Quantiles

In [ ]:
print("Card Features Quantiles:")
print(df_trans[['card1','card2','card3','card5']].quantile([0.01,0.025,.1,.25,.5,.75,.975,.99]))

We can see that Card 1 and Card 2 has a large distribution of values, so maybe it will be better to get the log of these columns.

In [ ]:
df_trans.loc[df_trans.card3.isin(df_trans.card3.value_counts()[df_trans.card3.value_counts()<200].index),'card3']="Others"
df_trans.loc[df_trans.card5.isin(df_trans.card5.value_counts()[df_trans.card5.value_counts()<300].index),'card5']="Others"

## Visualizing Card 1, Card 2 and Card 3 Distributions
* As Card 1 and 2 are numericals, I will plot the distribution of them.
* In Card 3, as we have many values with low frequencies, I decided to set value to "Others".
* Also, in Card 3, I set the % of Fraud ratio in yaxis2.

In [ ]:
tmp=pd.crosstab(df_trans['card3'],df_trans['isFraud'],normalize='index')*100
tmp=tmp.reset_index()
tmp.rename(columns={0:'NoFraud',1:'Fraud'},inplace=True)

tmp2=pd.crosstab(df_trans['card5'],df_trans['isFraud'],normalize='index')*100
tmp2=tmp2.reset_index()
tmp2.rename(columns={0:'NoFraud',1:'Fraud'},inplace=True)

plt.figure(figsize=(14,22))

plt.subplot(411)
g=sns.distplot(df_trans[df_trans['isFraud']==1]['card1'],label='Fraud')
g=sns.distplot(df_trans[df_trans['isFraud']==0]['card1'],label='NoFraud')
g.legend()
g.set_title("Card 1 Values Distribution by Target",fontsize=20)
g.set_xlabel("Card 1 Values",fontsize=18)
g.set_ylabel("Probability",fontsize=18)

In [ ]:
plt.subplot(412)
g1=sns.distplot(df_trans[df_trans['isFraud']==1]['card2'],label='Fraud')
g1=sns.distplot(df_trans[df_trans['isFraud']==0]['card2'],label='NoFraud')
g1.legend()
g1.set_title("Card 2 Values Distribution by Target",fontsize=20)
g1.set_xlabel("Card 2 Values",fontsize=18)
g1.set_ylabel("Probability",fontsize=18)

In [ ]:
plt.subplot(413)
g2 = sns.countplot(x='card3',data=df_trans,order=list(tmp.card3.values))
g22 = g2.twinx()
gg2 = sns.pointplot(x='card3',y='Fraud',data=tmp,color='black',order=list(tmp.card3.values))
gg2.set_ylabel("% of Fraud Transactions",fontsize=16)
g2.set_title("Card 3 Values Distribution and % of Transaction Frauds",fontsize=20)
g2.set_xlabel("Card 3 Values",fontsize=18)
g2.set_ylabel("Count",fontsize=18)
for p in g2.patches:
    height = p.get_height()
    g2.text(p.get_x()+p.get_width()/2,height + 25,'{:1.2f}%'.format(height/total*100),ha="center") 

In [ ]:
plt.subplot(414)
g3=sns.countplot(x='card5',data=df_trans,order=list(tmp2.card5.values))
g3t=g3.twinx()
g3t=sns.pointplot(x='card5',y='Fraud',data=tmp2,color='black',order=list(tmp2.card5.values))
g3t.set_ylabel("% of Fraud Transactions",fontsize=16)
g3.set_title("Card 5 Values Distribution and % of Transaction Frauds",fontsize=20)
g3.set_xticklabels(g3.get_xticklabels(),rotation=90)
g3.set_xlabel("Card 5 Values",fontsize=18)
g3.set_ylabel("Count",fontsize=18)
for p in g3.patches:
    height=p.get_height()
    g3.text(p.get_x()+p.get_width()/2,height+3,'{:1.2f}%'.format(height/total*100),ha='center',fontsize=11)
plt.subplots_adjust(hspace=0.6,top=0.85)

plt.show()

In Card 3, we can see that 150 and 185 are the most common values in the column.
We have over 12% of Fraud in 185 and about 3% in 150; The values with highest Fraud Transactions are 185 and 119.

In Card 5, the most frequent values are 226, 224 and 166 that represent 73% of data. Also it's possible to see high % of frauds in 137, 147 and 141 that has few entries for values.

## Card 4 - Categorical

In [ ]:
tmp=pd.crosstab(df_trans['card4'],df_trans['isFraud'],normalize='index')*100
tmp=tmp.reset_index()
tmp.rename(columns={0:"NoFraud",1:"Fraud"},inplace=True)

plt.figure(figsize=(14,10))
plt.suptitle("Card 4 Distribution",fontsize=22)

plt.subplot(221)
g=sns.countplot(x='card4',data=df_trans)
# plt.legend(title="Fraud",loc="upper center",labels=["No","Yes"])
g.set_title("Card 4 Distribution",fontsize=19)
g.set_ylim(0,420000)
g.set_xlabel("Card 4 Category Names",fontsize=17)
g.set_ylabel("Count",fontsize=17)
for p in g.patches:
    height=p.get_height()
    g.text(p.get_x()+p.get_width()/2,height+3,"{:1.2f}%".format(height/total*100),ha="center",fontsize=14)
    
plt.subplot(222)
g1=sns.countplot(x="card4",hue="isFraud",data=df_trans)
plt.legend(title="Fraud",loc="best",labels=["No","Yes"])
gt=g1.twinx()
gt=sns.pointplot(x='card4',y='Fraud',data=tmp,color='black',legend=False,order=['discover','mastercard','visa','american express'])
g1.set_ylabel("% of Fraud Transaction",fontsize=16)
g1.set_title("Card4 by Target(isFraud)",fontsize=19)
g1.set_xlabel("Card4 Category Names",fontsize=17)
g1.set_ylabel("Count",fontsize=17)

plt.subplot(212)
g3=sns.boxenplot(x='card4',y='TransactionAmt',hue='isFraud',data=df_trans[df_trans['TransactionAmt']<=2000])
g3.set_title("Card 4 Distribution by ProductCD and Target",fontsize=20)
g3.set_xlabel("Card 4 Category Names",fontsize=17)
g3.set_ylabel("Transaction Values",fontsize=17)

plt.subplots_adjust(hspace=0.6,top=0.85)

plt.show()

We can see that 97% of our data are in Mastercard(32%) and Visa(65%).

We have a highest value in discover(~8%) against ~3.5% of Mastercard and Visa and 2.87% in American Express.

## Card 6 - Categorical

In [ ]:
tmp=pd.crosstab(df_trans['card6'],df_trans['isFraud'],normalize='index')*100
tmp=tmp.reset_index()
tmp.rename(columns={0:"NoFraud",1:"Fraud"},inplace=True)

plt.figure(figsize=(14,10))
plt.suptitle('Card 6 Distributions',fontsize=22)

plt.subplot(221)
g=sns.countplot(x='card6',data=df_trans,order=list(tmp.card6.values))
# plt.legend(title="Fraud",loc='upper center',labels=['No','Yes'])
g.set_title("Card 6 Distribution",fontsize=19)
g.set_ylim(0,480000)
g.set_xlabel("Card 6 Category Names",fontsize=17)
g.set_ylabel("Count",fontsize=17)
for p in g.patches:
    height=p.get_height()
    g.text(p.get_x()+p.get_width()/2,height+3,"{:1.2f}%".format(height/total*100),ha='center',fontsize=14)
    
plt.subplot(222)
g1=sns.countplot(x='card6',hue='isFraud',data=df_trans,order=list(tmp.card6.values))
plt.legend(title='Fraud',loc='best',labels=['No','Yes'])
gt=g1.twinx()
g1=sns.pointplot(x='card6',y='Fraud',data=tmp,order=list(tmp.card6.values),color='black',legend=False)
gt.set_ylim(0,20)
g1.set_ylabel("% of Fraud Transactions",fontsize=16)
g1.set_title("Card 6 by Target(isFraud)",fontsize=19)
g1.set_xlabel("Card 6 Category Names",fontsize=17)
g1.set_ylabel("Count",fontsize=17)

plt.subplot(212)
g3=sns.boxenplot(x='card6',y='TransactionAmt',hue='isFraud',data=df_trans[df_trans['TransactionAmt']<=2000],order=list(tmp.card6.values))
g3.set_title("Card 6 Distribution by ProductCD and Target")
g3.set_xlabel("Card 6 Category Names",fontsize=17)
g3.set_ylabel("Transaction Values",fontsize=17)

plt.subplots_adjust(hspace=0.6,top=0.85)
plt.show()

All data is on Credit and Debit. We can see a high percentual of Frauds in Credit than Debit transactions.

The Distribution of Transaction Amoutn doesn't show clear differences.

## Exploring M1-M9 Features

In [ ]:
for col in ['M'+str(i) for i in range(1,10)]:
    df_trans[col]=df_trans[col].fillna('Miss')
    
def plotting_dist_ratio(df,col,lim=2000):
    tmp=pd.crosstab(df[col],df['isFraud'],normalize="index")*100
    tmp=tmp.reset_index()
    tmp.rename(columns={0:"NoFraud",1:"Fraud"},inplace=True)
    
    plt.figure(figsize=(20,5))
    plt.suptitle(f'{col} Distributions ',fontsize=22)
    
    plt.subplot(121)
    g=sns.countplot(x=col,data=df,order=list(df[col].values))
    # plt.legend(title="Fraud",loc="upper center",labels=['No','Yes'])
    g.set_title(f'{col} Distribution\nCount and % Fraud by each Category',fontsize=18)
    g.set_ylim(0,400000)
    gt=g.twinx()
    gt=sns.pointplot(x=col,y='Fraud',data=tmp,order=list(df[col].values),color="black",legend=False)
    gt.set_ylim(0,20)
    gt.set_ylabel("% of Fraud Transactions",fontsize=16)
    g.set_xlabel(f"{col} Category Names",fontsize=16)
    g.set_ylabel("Count",fontsize=17)
    for p in gt.patches:
        height=p.get_height()
        gt.text(p.get_x()+p.get_width()/2,height+3,"{:1.2f}%".format(height/total*100),ha='center',fontsize=14)
    
    perc_amt=(df_trans.groupby(['isFraud',col])['TransactionAmt'].sum()/total_amt*100).unstack('isFraud')
    perc_amt=per_amt.reset_index()
    perc_amt.rename(columns={0:"NoFraud",1:"Fraud"},inplace=True)
    
    plt.subplot(122)
    g1=sns.boxplot(x=col,y='TransactionAmt',hue='isFraud',data=df[df['TransactionAmt']<=lim],order=list(tmp[col].values))
    g1t=g1.twinx()
    g1t=sns.pointplot(x=col,y='Fraud',data=perc_amt,order=list(tmp[col].values))
    g1t.set_ylim(0,5)
    g1t.set_ylabel("% Fraud Total Amount",fontsize=16)
    g1.set_title(f"{col} by Transactions dist",fontsize=18)
    g1.set_xlabel(f"{col} Categorical Names",fontsize=16)
    g1.set_ylabel("Transaction Amount(U$)",fontsize=16)
    
    plt.subplots_adjust(hspace=.4,wspace=0.35,top=0.8)
    
    plt.show()

### M distributions : Count, % Fraud and Transaction Amount Distribution

In [ ]:
for col in ['M'+str(i) for i in range(1,10)]:
    plotting_dist_ratio(df_trans,col,lim=2500)

Only in M4 the Missing values isn't the highest % of Fraud.

### Addr1 and Addr2

In [ ]:
print("Card Features Quantiles: ")
print(df_trans[['addr1','addr2']].quantile([0.01,.025,.1,.25,.5,.75,.90,.975,.99]))

I will set all values in Addr1 that has less than 5000 entries to "Others".

In Addr2, I will set as "Others" all values with less than 50 entries.

In [ ]:
df_trans.loc[df_trans.addr1.isin(df_trans.addr1.value_counts()[df_trans.addr1.value_counts()<=5000].index),'addr1']="Others"
df_trans.loc[df_trans.addr2.isin(df_trans.addr2.value_counts()[df_trans.addr2.value_counts()<=50].index),'addr2']="Others"

### Addr1 Distributions

In [ ]:
def ploting_cnt_amt(df, col, lim=2000):
    tmp = pd.crosstab(df[col], df['isFraud'], normalize='index') * 100
    tmp = tmp.reset_index()
    tmp.rename(columns={0:'NoFraud', 1:'Fraud'}, inplace=True)
    
    plt.figure(figsize=(16,14))    
    plt.suptitle(f'{col} Distributions ', fontsize=24)
    
    plt.subplot(211)
    g = sns.countplot( x=col,  data=df, order=list(tmp[col].values))
    gt = g.twinx()
    gt = sns.pointplot(x=col, y='Fraud', data=tmp, order=list(tmp[col].values),
                       color='black', legend=False, )
    gt.set_ylim(0,tmp['Fraud'].max()*1.1)
    gt.set_ylabel("%Fraud Transactions", fontsize=16)
    g.set_title(f"Most Frequent {col} values and % Fraud Transactions", fontsize=20)
    g.set_xlabel(f"{col} Category Names", fontsize=16)
    g.set_ylabel("Count", fontsize=17)
    g.set_xticklabels(g.get_xticklabels(),rotation=45)
    sizes = []
    for p in g.patches:
        height = p.get_height()
        sizes.append(height)
        g.text(p.get_x()+p.get_width()/2.,
                height + 3,
                '{:1.2f}%'.format(height/total*100),
                ha="center",fontsize=12) 
        
    g.set_ylim(0,max(sizes)*1.15)
    
    perc_amt = (df.groupby(['isFraud',col])['TransactionAmt'].sum() \
                / df.groupby([col])['TransactionAmt'].sum() * 100).unstack('isFraud')
    perc_amt = perc_amt.reset_index()
    perc_amt.rename(columns={0:'NoFraud', 1:'Fraud'}, inplace=True)
    amt = df.groupby([col])['TransactionAmt'].sum().reset_index()
    perc_amt = perc_amt.fillna(0)
    plt.subplot(212)
    g1 = sns.barplot(x=col, y='TransactionAmt', 
                       data=amt, 
                       order=list(tmp[col].values))
    g1t = g1.twinx()
    g1t = sns.pointplot(x=col, y='Fraud', data=perc_amt, 
                        order=list(tmp[col].values),
                       color='black', legend=False, )
    g1t.set_ylim(0,perc_amt['Fraud'].max()*1.1)
    g1t.set_ylabel("%Fraud Total Amount", fontsize=16)
    g.set_xticklabels(g.get_xticklabels(),rotation=45)
    g1.set_title(f"{col} by Transactions Total + %of total and %Fraud Transactions", fontsize=20)
    g1.set_xlabel(f"{col} Category Names", fontsize=16)
    g1.set_ylabel("Transaction Total Amount(U$)", fontsize=16)
    g1.set_xticklabels(g.get_xticklabels(),rotation=45)    
    
    for p in g1.patches:
        height = p.get_height()
        g1.text(p.get_x()+p.get_width()/2.,
                height + 3,
                '{:1.2f}%'.format(height/total_amt*100),
                ha="center",fontsize=12) 
        
    plt.subplots_adjust(hspace=.4, top = 0.9)
    plt.show()

In [ ]:
ploting_cnt_amt(df_trans, 'addr1')

### Addr2 Distributions

In [ ]:
ploting_cnt_amt(df_trans, 'addr2')

Almost all entries in Addr2 are in the same value.

Interestingly in the value 65 , the percent of frauds is almost 60%.

Although the value 87 has 88% of total entries, it has 96% of Total Transaction Amounts.

### P emaildomain Distributions
* I will group all e-mail domains by the respective enterprises.
* Also, I wil lset as "Others" all values with less than 500 entries.

In [8]:
df_trans.loc[df_trans['P_emaildomain'].isin(['gmail.com', 'gmail']),'P_emaildomain'] = 'Google'

df_trans.loc[df_trans['P_emaildomain'].isin(['yahoo.com', 'yahoo.com.mx',  'yahoo.co.uk',
                                         'yahoo.co.jp', 'yahoo.de', 'yahoo.fr',
                                         'yahoo.es']), 'P_emaildomain'] = 'Yahoo Mail'
df_trans.loc[df_trans['P_emaildomain'].isin(['hotmail.com','outlook.com','msn.com', 'live.com.mx', 
                                         'hotmail.es','hotmail.co.uk', 'hotmail.de',
                                         'outlook.es', 'live.com', 'live.fr',
                                         'hotmail.fr']), 'P_emaildomain'] = 'Microsoft'
df_trans.loc[df_trans.P_emaildomain.isin(df_trans.P_emaildomain\
                                         .value_counts()[df_trans.P_emaildomain.value_counts() <= 500 ]\
                                         .index), 'P_emaildomain'] = "Others"
df_trans.P_emaildomain.fillna("NoInf", inplace=True)

### Plotting P-Email Domain

In [ ]:
ploting_cnt_amt(df_trans, 'P_emaildomain')

### R-Email Domain plot distribution
* I will group all e-mail domains by the respective enterprises.
* I will set as "Others" all values with less than 300 entries.

In [9]:
df_trans.loc[df_trans['R_emaildomain'].isin(['gmail.com', 'gmail']),'R_emaildomain'] = 'Google'

df_trans.loc[df_trans['R_emaildomain'].isin(['yahoo.com', 'yahoo.com.mx',  'yahoo.co.uk',
                                             'yahoo.co.jp', 'yahoo.de', 'yahoo.fr',
                                             'yahoo.es']), 'R_emaildomain'] = 'Yahoo Mail'
df_trans.loc[df_trans['R_emaildomain'].isin(['hotmail.com','outlook.com','msn.com', 'live.com.mx', 
                                             'hotmail.es','hotmail.co.uk', 'hotmail.de',
                                             'outlook.es', 'live.com', 'live.fr',
                                             'hotmail.fr']), 'R_emaildomain'] = 'Microsoft'
df_trans.loc[df_trans.R_emaildomain.isin(df_trans.R_emaildomain\
                                         .value_counts()[df_trans.R_emaildomain.value_counts() <= 300 ]\
                                         .index), 'R_emaildomain'] = "Others"
df_trans.R_emaildomain.fillna("NoInf", inplace=True)

In [ ]:
ploting_cnt_amt(df_trans, 'R_emaildomain')

We can see a very similiar distribution in both email and domain features.

### C1-C14 features

In [13]:
resumetable(df_trans[['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8','C9', 'C10', 'C11', 'C12', 'C13', 'C14']])

Dataset Shape: (590540, 14)


,Name,dtypes,Missing,Uniques,First Value,Second Value,Third Value,Entropy
0,C1,float16,0,1495,1.0,1.0,1.0,2.72
1,C2,float16,0,1167,1.0,1.0,1.0,2.75
2,C3,float16,0,27,0.0,0.0,0.0,0.04
3,C4,float16,0,1223,0.0,0.0,0.0,1.12
4,C5,float16,0,319,0.0,0.0,0.0,2.06
5,C6,float16,0,1291,1.0,1.0,1.0,2.52
6,C7,float16,0,1069,0.0,0.0,0.0,0.71
7,C8,float16,0,1130,0.0,0.0,0.0,1.25
8,C9,float16,0,205,1.0,0.0,1.0,2.62
9,C10,float16,0,1122,0.0,0.0,0.0,1.23


In [12]:
df_trans[['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8','C9', 'C10', 'C11', 'C12', 'C13', 'C14']].describe()

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14
count,590540.0,590540.0,590540.0,590540.0,590540.0,590540.0,590540.0,590540.0,590540.0,590540.0,590540.0,590540.0,590540.0,590540.0
mean,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
50%,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,3.0,1.0
75%,3.0,3.0,0.0,0.0,1.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,12.0,2.0
max,4684.0,5692.0,26.0,2252.0,349.0,2252.0,2256.0,3332.0,210.0,3256.0,3188.0,3188.0,2918.0,1429.0


In [14]:
df_trans.loc[df_trans.C1.isin(df_trans.C1.value_counts()[df_trans.C1.value_counts() <= 400 ].index), 'C1'] = "Others"

### C1 Distribution Plot

In [ ]:
ploting_cnt_amt(df_trans, 'C1')

In [15]:
df_trans.loc[df_trans.C2.isin(df_trans.C2.value_counts()[df_trans.C2.value_counts() <= 350 ].index), 'C2'] = "Others"

In [ ]:
ploting_cnt_amt(df_trans, 'C2')

### TimeDelta Feature
* Let's see if the frauds have some specific hour that has highest % of frauds.

In [16]:
import datetime

START_DATE = '2017-12-01'
startdate = datetime.datetime.strptime(START_DATE, "%Y-%m-%d")
df_trans["Date"] = df_trans['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds=x)))

df_trans['_Weekdays'] = df_trans['Date'].dt.dayofweek
df_trans['_Hours'] = df_trans['Date'].dt.hour
df_trans['_Days'] = df_trans['Date'].dt.day

In [ ]:
ploting_cnt_amt(df_trans, '_Days')

### Plotting Weekdays Distributions

In [ ]:
ploting_cnt_amt(df_trans, '_Weekdays')

### Plotting Hours Distributions

In [ ]:
ploting_cnt_amt(df_trans, '_Hours')

### Modelling

In [ ]:
df_trans = pd.read_csv('../input/train_transaction.csv')
df_test_trans = pd.read_csv('../input/test_transaction.csv')

df_id = pd.read_csv('../input/train_identity.csv')
df_test_id = pd.read_csv('../input/test_identity.csv')

sample_submission = pd.read_csv('../input/sample_submission.csv', index_col='TransactionID')

df_train = df_trans.merge(df_id, how='left', left_index=True, right_index=True, on='TransactionID')
df_test = df_test_trans.merge(df_test_id, how='left', left_index=True, right_index=True, on='TransactionID')

print(df_train.shape)
print(df_test.shape)

In [ ]:
df_train = reduce_mem_usage(df_train)
df_test = reduce_mem_usage(df_test)

In [ ]:
emails = {'gmail': 'google', 'att.net': 'att', 'twc.com': 'spectrum', 
          'scranton.edu': 'other', 'optonline.net': 'other', 'hotmail.co.uk': 'microsoft',
          'comcast.net': 'other', 'yahoo.com.mx': 'yahoo', 'yahoo.fr': 'yahoo',
          'yahoo.es': 'yahoo', 'charter.net': 'spectrum', 'live.com': 'microsoft', 
          'aim.com': 'aol', 'hotmail.de': 'microsoft', 'centurylink.net': 'centurylink',
          'gmail.com': 'google', 'me.com': 'apple', 'earthlink.net': 'other', 'gmx.de': 'other',
          'web.de': 'other', 'cfl.rr.com': 'other', 'hotmail.com': 'microsoft', 
          'protonmail.com': 'other', 'hotmail.fr': 'microsoft', 'windstream.net': 'other', 
          'outlook.es': 'microsoft', 'yahoo.co.jp': 'yahoo', 'yahoo.de': 'yahoo',
          'servicios-ta.com': 'other', 'netzero.net': 'other', 'suddenlink.net': 'other',
          'roadrunner.com': 'other', 'sc.rr.com': 'other', 'live.fr': 'microsoft',
          'verizon.net': 'yahoo', 'msn.com': 'microsoft', 'q.com': 'centurylink', 
          'prodigy.net.mx': 'att', 'frontier.com': 'yahoo', 'anonymous.com': 'other', 
          'rocketmail.com': 'yahoo', 'sbcglobal.net': 'att', 'frontiernet.net': 'yahoo', 
          'ymail.com': 'yahoo', 'outlook.com': 'microsoft', 'mail.com': 'other', 
          'bellsouth.net': 'other', 'embarqmail.com': 'centurylink', 'cableone.net': 'other', 
          'hotmail.es': 'microsoft', 'mac.com': 'apple', 'yahoo.co.uk': 'yahoo', 'netzero.com': 'other', 
          'yahoo.com': 'yahoo', 'live.com.mx': 'microsoft', 'ptd.net': 'other', 'cox.net': 'other',
          'aol.com': 'aol', 'juno.com': 'other', 'icloud.com': 'apple'}

us_emails = ['gmail', 'net', 'edu']

for c in ['P_emaildomain', 'R_emaildomain']:
    df_train[c + '_bin'] = df_train[c].map(emails)
    df_test[c + '_bin'] = df_test[c].map(emails)
    
    df_train[c + '_suffix'] = df_train[c].map(lambda x: str(x).split('.')[-1])
    df_test[c + '_suffix'] = df_test[c].map(lambda x: str(x).split('.')[-1])
    
    df_train[c + '_suffix'] = df_train[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')
    df_test[c + '_suffix'] = df_test[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')

In [ ]:
for f in df_train.drop('isFraud', axis=1).columns:
    if df_train[f].dtype=='object' or df_test[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(df_train[f].values) + list(df_test[f].values))
        df_train[f] = lbl.transform(list(df_train[f].values))
        df_test[f] = lbl.transform(list(df_test[f].values))   

In [ ]:
df_train['Trans_min_mean'] = df_train['TransactionAmt'] - df_train['TransactionAmt'].mean()
df_train['Trans_min_std'] = df_train['Trans_min_mean'] / df_train['TransactionAmt'].std()
df_test['Trans_min_mean'] = df_test['TransactionAmt'] - df_test['TransactionAmt'].mean()
df_test['Trans_min_std'] = df_test['Trans_min_mean'] / df_test['TransactionAmt'].std()

In [ ]:
df_train['TransactionAmt_to_mean_card1'] = df_train['TransactionAmt'] / df_train.groupby(['card1'])['TransactionAmt'].transform('mean')
df_train['TransactionAmt_to_mean_card4'] = df_train['TransactionAmt'] / df_train.groupby(['card4'])['TransactionAmt'].transform('mean')
df_train['TransactionAmt_to_std_card1'] = df_train['TransactionAmt'] / df_train.groupby(['card1'])['TransactionAmt'].transform('std')
df_train['TransactionAmt_to_std_card4'] = df_train['TransactionAmt'] / df_train.groupby(['card4'])['TransactionAmt'].transform('std')

df_test['TransactionAmt_to_mean_card1'] = df_test['TransactionAmt'] / df_test.groupby(['card1'])['TransactionAmt'].transform('mean')
df_test['TransactionAmt_to_mean_card4'] = df_test['TransactionAmt'] / df_test.groupby(['card4'])['TransactionAmt'].transform('mean')
df_test['TransactionAmt_to_std_card1'] = df_test['TransactionAmt'] / df_test.groupby(['card1'])['TransactionAmt'].transform('std')
df_test['TransactionAmt_to_std_card4'] = df_test['TransactionAmt'] / df_test.groupby(['card4'])['TransactionAmt'].transform('std')

In [ ]:
df_train['TransactionAmt'] = np.log(df_train['TransactionAmt'])
df_test['TransactionAmt'] = np.log(df_test['TransactionAmt'])

In [ ]:
df_test['isFraud'] = 'test'
df = pd.concat([df_train, df_test], axis=0, sort=False )
df = df.reset_index()
df = df.drop('index', axis=1)

In [ ]:
def PCA_change(df, cols, n_components, prefix='PCA_', rand_seed=4):
    pca = PCA(n_components=n_components, random_state=rand_seed)

    principalComponents = pca.fit_transform(df[cols])

    principalDf = pd.DataFrame(principalComponents)

    df.drop(cols, axis=1, inplace=True)

    principalDf.rename(columns=lambda x: str(prefix)+str(x), inplace=True)

    df = pd.concat([df, principalDf], axis=1)
    
    return df

In [ ]:
mas_v = df_train.columns[55:394]

In [ ]:
from sklearn.preprocessing import minmax_scale
from sklearn.decomposition import PCA
# from sklearn.cluster import KMeans

for col in mas_v:
    df[col] = df[col].fillna((df[col].min() - 2))
    df[col] = (minmax_scale(df[col], feature_range=(0,1)))

    
df = PCA_change(df, mas_v, prefix='PCA_V_', n_components=30)

In [ ]:
df = reduce_mem_usage(df)

In [ ]:
df_train, df_test = df[df['isFraud'] != 'test'], df[df['isFraud'] == 'test'].drop('isFraud', axis=1)

In [ ]:
df_train.shape()

In [ ]:
X_train = df_train.sort_values('TransactionDT').drop(['isFraud', 'TransactionDT',#'Card_ID'
                                                     ],axis=1)
y_train = df_train.sort_values('TransactionDT')['isFraud'].astype(bool)

X_test = df_test.sort_values('TransactionDT').drop(['TransactionDT',
                                                    #'Card_ID'
                                                   ],axis=1)
del df_train
df_test = df_test[["TransactionDT"]]

In [ ]:
from sklearn.model_selection import KFold,TimeSeriesSplit
from sklearn.metrics import roc_auc_score
from xgboost import plot_importance
from sklearn.metrics import make_scorer

import time
def objective(params):
    time1 = time.time()
    params = {
        'max_depth': int(params['max_depth']),
        'gamma': "{:.3f}".format(params['gamma']),
        'subsample': "{:.2f}".format(params['subsample']),
        'reg_alpha': "{:.3f}".format(params['reg_alpha']),
        'reg_lambda': "{:.3f}".format(params['reg_lambda']),
        'learning_rate': "{:.3f}".format(params['learning_rate']),
        'num_leaves': '{:.3f}'.format(params['num_leaves']),
        'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
        'min_child_samples': '{:.3f}'.format(params['min_child_samples']),
        'feature_fraction': '{:.3f}'.format(params['feature_fraction']),
        'bagging_fraction': '{:.3f}'.format(params['bagging_fraction'])
    }

    print("\n############## New Run ################")
    print(f"params = {params}")
    FOLDS = 7
    count=1
    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)

    tss = TimeSeriesSplit(n_splits=FOLDS)
    y_preds = np.zeros(sample_submission.shape[0])
    y_oof = np.zeros(X_train.shape[0])
    score_mean = 0
    for tr_idx, val_idx in tss.split(X_train, y_train):
        clf = xgb.XGBClassifier(
            n_estimators=600, random_state=4, verbose=True, 
            tree_method='gpu_hist', 
            **params
        )

        X_tr, X_vl = X_train.iloc[tr_idx, :], X_train.iloc[val_idx, :]
        y_tr, y_vl = y_train.iloc[tr_idx], y_train.iloc[val_idx]
        
        clf.fit(X_tr, y_tr)
        #y_pred_train = clf.predict_proba(X_vl)[:,1]
        #print(y_pred_train)
        score = make_scorer(roc_auc_score, needs_proba=True)(clf, X_vl, y_vl)
        # plt.show()
        score_mean += score
        print(f'{count} CV - score: {round(score, 4)}')
        count += 1
    time2 = time.time() - time1
    print(f"Total Time Run: {round(time2 / 60,2)}")
    gc.collect()
    print(f'Mean ROC_AUC: {score_mean / FOLDS}')
    del X_tr, X_vl, y_tr, y_vl, clf, score
    return -(score_mean / FOLDS)


space = {
    # The maximum depth of a tree, same as GBM.
    # Used to control over-fitting as higher depth will allow model 
    # to learn relations very specific to a particular sample.
    # Should be tuned using CV.
    # Typical values: 3-10
    'max_depth': hp.quniform('max_depth', 7, 23, 1),
    
    # reg_alpha: L1 regularization term. L1 regularization encourages sparsity 
    # (meaning pulling weights to 0). It can be more useful when the objective
    # is logistic regression since you might need help with feature selection.
    'reg_alpha':  hp.uniform('reg_alpha', 0.01, 0.4),
    
    # reg_lambda: L2 regularization term. L2 encourages smaller weights, this
    # approach can be more useful in tree-models where zeroing 
    # features might not make much sense.
    'reg_lambda': hp.uniform('reg_lambda', 0.01, .4),
    
    # eta: Analogous to learning rate in GBM
    # Makes the model more robust by shrinking the weights on each step
    # Typical final values to be used: 0.01-0.2
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    
    # colsample_bytree: Similar to max_features in GBM. Denotes the 
    # fraction of columns to be randomly samples for each tree.
    # Typical values: 0.5-1
    'colsample_bytree': hp.uniform('colsample_bytree', 0.3, .9),
    
    # A node is split only when the resulting split gives a positive
    # reduction in the loss function. Gamma specifies the 
    # minimum loss reduction required to make a split.
    # Makes the algorithm conservative. The values can vary depending on the loss function and should be tuned.
    'gamma': hp.uniform('gamma', 0.01, .7),
    
    # more increases accuracy, but may lead to overfitting.
    # num_leaves: the number of leaf nodes to use. Having a large number 
    # of leaves will improve accuracy, but will also lead to overfitting.
    'num_leaves': hp.choice('num_leaves', list(range(20, 250, 10))),
    
    # specifies the minimum samples per leaf node.
    # the minimum number of samples (data) to group into a leaf. 
    # The parameter can greatly assist with overfitting: larger sample
    # sizes per leaf will reduce overfitting (but may lead to under-fitting).
    'min_child_samples': hp.choice('min_child_samples', list(range(100, 250, 10))),
    
    # subsample: represents a fraction of the rows (observations) to be 
    # considered when building each subtree. Tianqi Chen and Carlos Guestrin
    # in their paper A Scalable Tree Boosting System recommend 
    'subsample': hp.choice('subsample', [0.2, 0.4, 0.5, 0.6, 0.7, .8, .9]),
    
    # randomly select a fraction of the features.
    # feature_fraction: controls the subsampling of features used
    # for training (as opposed to subsampling the actual training data in 
    # the case of bagging). Smaller fractions reduce overfitting.
    'feature_fraction': hp.uniform('feature_fraction', 0.4, .8),
    
    # randomly bag or subsample training data.
    'bagging_fraction': hp.uniform('bagging_fraction', 0.4, .9)
    
    # bagging_fraction and bagging_freq: enables bagging (subsampling) 
    # of the training data. Both values need to be set for bagging to be used.
    # The frequency controls how often (iteration) bagging is used. Smaller
    # fractions and frequencies reduce overfitting.
}

In [ ]:
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=27)

# Print best parameters
best_params = space_eval(space, best)

In [ ]:
print("BEST PARAMS: ", best_params)

best_params['max_depth'] = int(best_params['max_depth'])

In [ ]:
clf = xgb.XGBClassifier(
    n_estimators=300,
    **best_params,
    tree_method='gpu_hist'
)

clf.fit(X_train, y_train)

y_preds = clf.predict_proba(X_test)[:,1] 

In [ ]:
feature_important = clf.get_booster().get_score(importance_type="weight")
keys = list(feature_important.keys())
values = list(feature_important.values())

data = pd.DataFrame(data=values, index=keys, columns=["score"]).sort_values(by = "score", ascending=False)

# Top 10 features
data.head(20)

In [ ]:
sample_submission['isFraud'] = y_preds
sample_submission.to_csv('XGB_hypopt_model.csv')